# Summary
- mincorr and minlinreg are similar. maxcorr and maxlinreg are similar. 
- spearman among different layers is lower (even goes negative) in lower layers for:
    - bert_large
    - openai
    - bert_base
  
  but not true for elmo or calypso
- intermediate layers appear most similar to the next and previous layer of the same model
- lower layers (eg. in bert) are more correlated with lower layers in other models

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from os.path import basename, dirname
import torch

In [2]:
# Set `res_d`, `network_l`, `num_neurons_d`
base = "/data/sls/temp/johnmwu/contextual-corr-analysis/results7_"
res_fname = {method : base + method for method in 
                {"maxcorr", "mincorr", "maxlinreg", "minlinreg", "cca", "lincka"}}

res_d = {}
res_d["maxcorr"] = json.load(open(res_fname["maxcorr"], "r"))
res_d["mincorr"] = json.load(open(res_fname["mincorr"], "r"))
res_d["maxlinreg"] = json.load(open(res_fname["maxlinreg"], "r"))
res_d["minlinreg"] = json.load(open(res_fname["minlinreg"], "r"))

network_l = [network for network in res_d["maxcorr"]]

num_neurons_d = {}
for network in network_l:
    num_neurons_d[network] = len(res_d["maxcorr"][network])

# MaxCorr, MinCorr, MaxLinReg, MinLinReg

### Sorts

In [3]:
rk_methods = {"maxcorr", "mincorr", "maxlinreg", "minlinreg"}

In [4]:
neuron_sorts = {network: {} for network in network_l}
for network in network_l:
    for method in rk_methods:
        neuron_sorts[network][method] = [neuron[0] for neuron in res_d[method][network]]

In [5]:
# Set `rk_ar_d`, `rk_df_d`, `spearman_d`
rk_ar_d = {}
rk_df_d = {}
spearman_d = {}
for network in network_l:
    # rk_ar[method_ix, neuron] = rank
    # need to invert the permutation
    rk_ar = np.stack([
            [neuron_sorts[network][method].index(neuron) for neuron in range(len(neuron_sorts[network][method]))]
                      for method in rk_methods
    ]).T
    rk_df = pd.DataFrame(rk_ar, columns=list(rk_methods))
    
    rk_ar_d[network] = rk_ar
    rk_df_d[network] = rk_df
    spearman_d[network] = rk_df.corr()

In [6]:
for network in network_l:
    print(network)
    # print(rk_df_d[network])
    print(spearman_d[network])

bert_large_cased_0
            maxcorr  maxlinreg  minlinreg   mincorr
maxcorr    1.000000   0.648555  -0.177012 -0.217173
maxlinreg  0.648555   1.000000   0.281875  0.246626
minlinreg -0.177012   0.281875   1.000000  0.812337
mincorr   -0.217173   0.246626   0.812337  1.000000
bert_large_cased_1
            maxcorr  maxlinreg  minlinreg   mincorr
maxcorr    1.000000   0.966645   0.253680  0.049593
maxlinreg  0.966645   1.000000   0.314057  0.107344
minlinreg  0.253680   0.314057   1.000000  0.695416
mincorr    0.049593   0.107344   0.695416  1.000000
bert_large_cased_2
            maxcorr  maxlinreg  minlinreg   mincorr
maxcorr    1.000000   0.977872   0.468780  0.183748
maxlinreg  0.977872   1.000000   0.482464  0.196051
minlinreg  0.468780   0.482464   1.000000  0.727491
mincorr    0.183748   0.196051   0.727491  1.000000
bert_large_cased_3
            maxcorr  maxlinreg  minlinreg   mincorr
maxcorr    1.000000   0.980595   0.373058  0.119859
maxlinreg  0.980595   1.000000   0.39966

### Distributions

In [7]:
for network in network_l:
    print("\n\n" + network + ":")
    for method in rk_methods:
        print("\n" + method)
        corr_ar = np.stack([
            list(notated_neuron[1].values())
            for notated_neuron in res_d[method][network]
        ])
        corr_df = pd.DataFrame(np.abs(corr_ar),
                              columns=[net for net in network_l if net != network])
        
        print("\nmaxcounts")
        print(corr_df.idxmax(axis=1).value_counts())
        
        print("\nmincounts")
        print(corr_df.idxmin(axis=1).value_counts())
        
        print("\ncorr_df_pctl")
        corr_df_pctl = pd.DataFrame()
        corr_df_pctl["max"] = corr_df.max(axis=1)
        corr_df_pctl["min"] = corr_df.min(axis=1)
        corr_df_pctl["med"] = corr_df.median(axis=1)
        print(corr_df_pctl.rank().corr()) # spearman
        
#         corr_df.plot()
#         plt.xlabel("neuron (sorted order)")
#         plt.ylabel("correlation")
#         plt.show()
        
#         plt.hist(corr_ar.ravel(), bins=100)
#         plt.xlabel("correlation")
#         plt.ylabel("count")
#         plt.show()



bert_large_cased_0:

maxcorr

maxcounts
bert_large_cased_1                  981
bert_base_cased_0                    40
calypso_transformer_6_512_base_0      2
openai_transformer_2                  1
dtype: int64

mincounts
elmo_4x4096_512_2                   281
elmo_4x4096_512_3                   111
bert_large_cased_22                  65
bert_large_cased_21                  58
bert_large_cased_23                  46
bert_base_cased_10                   39
openai_transformer_12                39
elmo_4x4096_512_1                    37
calypso_transformer_6_512_base_6     29
elmo_4x4096_512_4                    29
elmo_original_1                      25
bert_base_cased_11                   25
bert_large_cased_18                  23
bert_large_cased_19                  22
elmo_original_2                      20
bert_base_cased_12                   17
bert_base_cased_9                    15
bert_large_cased_17                  12
openai_transformer_11                11
bert_base_case

          max       min       med
max  1.000000  0.183748  0.314837
min  0.183748  1.000000  0.855940
med  0.314837  0.855940  1.000000

maxlinreg

maxcounts
bert_large_cased_3    1024
dtype: int64

mincounts
openai_transformer_12               970
calypso_transformer_6_512_base_0     35
elmo_4x4096_512_0                    14
elmo_original_0                       2
elmo_4x4096_512_4                     1
openai_transformer_0                  1
openai_transformer_11                 1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.482464  0.729536
min  0.482464  1.000000  0.864348
med  0.729536  0.864348  1.000000

minlinreg

maxcounts
bert_large_cased_3    1024
dtype: int64

mincounts
openai_transformer_12               970
calypso_transformer_6_512_base_0     35
elmo_4x4096_512_0                    14
elmo_original_0                       2
elmo_4x4096_512_4                     1
openai_transformer_0                  1
openai_transformer_11              

          max       min       med
max  1.000000  0.439943  0.731771
min  0.439943  1.000000  0.863522
med  0.731771  0.863522  1.000000

minlinreg

maxcounts
bert_large_cased_5    1024
dtype: int64

mincounts
openai_transformer_12               978
calypso_transformer_6_512_base_0     33
elmo_4x4096_512_0                     7
elmo_original_0                       4
elmo_4x4096_512_4                     1
openai_transformer_0                  1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.439943  0.731771
min  0.439943  1.000000  0.863522
med  0.731771  0.863522  1.000000

mincorr

maxcounts
bert_large_cased_5    740
bert_large_cased_3    284
dtype: int64

mincounts
elmo_4x4096_512_2                   229
elmo_4x4096_512_3                   152
elmo_original_2                      60
bert_base_cased_12                   59
openai_transformer_12                51
calypso_transformer_6_512_base_6     46
elmo_4x4096_512_4                    42
bert_large_c


maxcounts
bert_large_cased_7                  780
bert_large_cased_5                  243
calypso_transformer_6_512_base_0      1
dtype: int64

mincounts
elmo_4x4096_512_2                   202
elmo_4x4096_512_3                   134
elmo_original_2                      63
elmo_original_0                      54
openai_transformer_12                53
elmo_4x4096_512_4                    52
calypso_transformer_6_512_base_6     50
bert_base_cased_12                   48
elmo_4x4096_512_0                    42
elmo_4x4096_512_1                    35
elmo_original_1                      31
openai_transformer_0                 30
bert_large_cased_23                  27
bert_large_cased_24                  25
bert_base_cased_11                   21
bert_base_cased_10                   17
bert_large_cased_21                  14
bert_large_cased_22                  13
openai_transformer_1                 13
openai_transformer_6                 12
openai_transformer_11                11
calyp

bert_large_cased_10                 668
bert_large_cased_8                  355
calypso_transformer_6_512_base_0      1
dtype: int64

mincounts
elmo_4x4096_512_2                   200
elmo_4x4096_512_3                   155
openai_transformer_12                75
elmo_original_0                      65
elmo_original_2                      64
elmo_4x4096_512_4                    55
elmo_4x4096_512_0                    46
calypso_transformer_6_512_base_6     41
bert_base_cased_12                   38
openai_transformer_0                 34
elmo_4x4096_512_1                    32
elmo_original_1                      23
openai_transformer_1                 21
calypso_transformer_6_512_base_0     19
openai_transformer_11                16
bert_base_cased_11                   14
bert_large_cased_24                  11
calypso_transformer_6_512_base_5     10
openai_transformer_6                 10
openai_transformer_3                  9
openai_transformer_9                  9
openai_transform

...

bert_large_cased_24                 75
bert_base_cased_12                  59
bert_large_cased_23                 59
bert_large_cased_22                 54
openai_transformer_12               54
bert_large_cased_8                  50
bert_large_cased_21                 41
elmo_4x4096_512_2                   41
bert_large_cased_5                  40
bert_large_cased_19                 35
bert_large_cased_9                  35
elmo_original_0                     33
elmo_4x4096_512_3                   33
bert_base_cased_10                  32
bert_large_cased_18                 30
bert_large_cased_7                  26
openai_transformer_5                25
bert_large_cased_20                 23
elmo_4x4096_512_4                   22
elmo_original_2                     21
bert_base_cased_9                   20
bert_large_cased_2                  19
elmo_4x4096_512_0                   18
bert_large_cased_15                 13
bert_base_cased_11                  11
openai_transformer_2     

calypso_transformer_6_512_base_4    1003
calypso_transformer_6_512_base_2      21
dtype: int64

mincounts
openai_transformer_12               360
elmo_4x4096_512_0                   304
elmo_original_0                     170
bert_large_cased_24                 143
openai_transformer_0                 25
openai_transformer_5                 10
openai_transformer_1                  6
bert_base_cased_12                    3
calypso_transformer_6_512_base_0      3
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.323251  0.864557
min  0.323251  1.000000  0.557188
med  0.864557  0.557188  1.000000

mincorr

maxcounts
calypso_transformer_6_512_base_2    651
calypso_transformer_6_512_base_4    373
dtype: int64

mincounts
bert_large_cased_23                 72
bert_large_cased_24                 69
elmo_original_0                     64
bert_large_cased_8                  57
bert_large_cased_5                  52
bert_large_cased_22                 44
elmo_4x4096_5

          max       min       med
max  1.000000  0.342638  0.521079
min  0.342638  1.000000  0.821827
med  0.521079  0.821827  1.000000


calypso_transformer_6_512_base_6:

maxcorr

maxcounts
calypso_transformer_6_512_base_5    1023
calypso_transformer_6_512_base_4       1
dtype: int64

mincounts
elmo_original_0                     166
elmo_4x4096_512_0                   118
bert_large_cased_23                  54
bert_large_cased_24                  46
bert_large_cased_8                   44
bert_large_cased_22                  42
bert_base_cased_12                   36
bert_large_cased_7                   35
bert_large_cased_2                   29
openai_transformer_12                27
bert_large_cased_21                  25
bert_large_cased_5                   23
bert_large_cased_6                   22
openai_transformer_0                 20
bert_large_cased_19                  19
bert_base_cased_3                    17
bert_large_cased_10                  17
bert_large_cased_1    


maxcounts
elmo_4x4096_512_2    1018
elmo_original_1         6
dtype: int64

mincounts
openai_transformer_12               507
openai_transformer_0                297
bert_large_cased_24                 181
bert_base_cased_12                   19
calypso_transformer_6_512_base_0     12
openai_transformer_1                  4
bert_base_cased_0                     2
openai_transformer_5                  2
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.608247  0.834820
min  0.608247  1.000000  0.840016
med  0.834820  0.840016  1.000000

minlinreg

maxcounts
elmo_4x4096_512_2    1018
elmo_original_1         6
dtype: int64

mincounts
openai_transformer_12               507
openai_transformer_0                297
bert_large_cased_24                 181
bert_base_cased_12                   19
calypso_transformer_6_512_base_0     12
openai_transformer_1                  4
bert_base_cased_0                     2
openai_transformer_5                  2
dtype: int64


maxcounts
elmo_4x4096_512_4    1020
elmo_4x4096_512_2       4
dtype: int64

mincounts
openai_transformer_0                512
openai_transformer_12               464
calypso_transformer_6_512_base_0     17
openai_transformer_1                 14
elmo_original_0                       9
openai_transformer_5                  6
bert_large_cased_24                   1
elmo_4x4096_512_0                     1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.796562  0.855326
min  0.796562  1.000000  0.942786
med  0.855326  0.942786  1.000000

mincorr

maxcounts
elmo_4x4096_512_4    966
elmo_4x4096_512_2     58
dtype: int64

mincounts
openai_transformer_0                88
bert_large_cased_5                  70
openai_transformer_12               62
bert_large_cased_2                  45
bert_large_cased_4                  41
bert_large_cased_24                 40
bert_large_cased_3                  39
bert_base_cased_1                   35
bert_base_cased_12      

In [8]:
network = "calypso_transformer_6_512_base_0"
method = "maxcorr"
print("\n\n" + network + ":")

print("\n" + method)
corr_ar = np.stack([
    list(notated_neuron[1].values())
    for notated_neuron in res_d[method][network]
])
corr_df = pd.DataFrame(np.abs(corr_ar),
                      columns=[net for net in network_l if net != network])

print("\nmaxcounts")
print(corr_df.idxmax(axis=1).value_counts())

print("\nmincounts")
print(corr_df.idxmin(axis=1).value_counts())

print("\ncorr_df_pctl")
corr_df_pctl = pd.DataFrame()
corr_df_pctl["max"] = corr_df.max(axis=1)
corr_df_pctl["min"] = corr_df.min(axis=1)
corr_df_pctl["med"] = corr_df.median(axis=1)
print(corr_df_pctl.rank().corr()) # spearman



calypso_transformer_6_512_base_0:

maxcorr

maxcounts
calypso_transformer_6_512_base_1    916
elmo_original_1                      34
calypso_transformer_6_512_base_2     22
calypso_transformer_6_512_base_4     10
bert_large_cased_20                   6
calypso_transformer_6_512_base_6      6
calypso_transformer_6_512_base_3      6
openai_transformer_2                  6
elmo_4x4096_512_0                     4
openai_transformer_0                  2
openai_transformer_4                  2
bert_large_cased_5                    2
bert_large_cased_2                    2
bert_large_cased_0                    2
bert_base_cased_1                     2
bert_large_cased_11                   2
dtype: int64

mincounts
bert_base_cased_12                  138
bert_large_cased_24                 128
openai_transformer_12                78
bert_large_cased_22                  70
bert_base_cased_10                   64
bert_base_cased_11                   60
bert_large_cased_23                  60


In [9]:
corr_df

,bert_large_cased_0,bert_large_cased_1,bert_large_cased_2,bert_large_cased_3,bert_large_cased_4,bert_large_cased_5,bert_large_cased_6,bert_large_cased_7,bert_large_cased_8,bert_large_cased_9,...,calypso_transformer_6_512_base_2,calypso_transformer_6_512_base_3,calypso_transformer_6_512_base_4,calypso_transformer_6_512_base_5,calypso_transformer_6_512_base_6,elmo_4x4096_512_0,elmo_4x4096_512_1,elmo_4x4096_512_2,elmo_4x4096_512_3,elmo_4x4096_512_4
0,1.092270,0.581859,0.630460,0.641075,0.765226,0.942053,0.995977,0.992174,1.062185,1.018505,...,0.599399,0.596224,0.591232,0.564971,0.723748,0.484699,0.352679,0.369379,0.355852,0.409234
1,1.092270,0.581859,0.630460,0.641075,0.765226,0.942053,0.995977,0.992174,1.062185,1.018505,...,0.599399,0.596224,0.591232,0.564971,0.723748,0.484699,0.352679,0.369379,0.355852,0.409234
2,0.379928,0.351945,0.343969,0.358741,0.365370,0.366556,0.362762,0.370998,0.362582,0.353839,...,0.572167,0.568370,0.561055,0.525884,0.413914,0.436640,0.334435,0.329261,0.327309,0.304382
3,0.379928,0.351945,0.343969,0.358741,0.365370,0.366556,0.362762,0.370998,0.362582,0.353839,...,0.572167,0.568370,0.561055,0.525884,0.413914,0.436640,0.334435,0.329261,0.327309,0.304382
4,0.320125,0.275337,0.262605,0.252768,0.242704,0.243742,0.259069,0.255931,0.228539,0.231637,...,0.822608,0.752733,0.630810,0.519475,0.361417,0.490976,0.249035,0.218912,0.362929,0.473615
5,0.320125,0.275337,0.262605,0.252768,0.242704,0.243742,0.259069,0.255931,0.228539,0.231637,...,0.822608,0.752733,0.630810,0.519475,0.361417,0.490976,0.249035,0.218912,0.362929,0.473615
6,0.605103,0.666045,0.644904,0.652232,0.646065,0.662751,0.681597,0.674260,0.651238,0.639640,...,0.846168,0.768234,0.707586,0.711004,0.592584,0.538621,0.382261,0.436649,0.506820,0.564428
7,0.605103,0.666045,0.644904,0.652232,0.646065,0.662751,0.681597,0.674260,0.651238,0.639640,...,0.846168,0.768234,0.707586,0.711004,0.592584,0.538621,0.382261,0.436649,0.506820,0.564428
8,0.422056,0.345187,0.316551,0.322967,0.296497,0.303741,0.330338,0.324257,0.318767,0.304707,...,0.842490,0.760641,0.716792,0.703909,0.514605,0.694479,0.503767,0.526192,0.532208,0.568026
9,0.422056,0.345187,0.316551,0.322967,0.296497,0.303741,0.330338,0.324257,0.318767,0.304707,...,0.842490,0.760641,0.716792,0.703909,0.514605,0.694479,0.503767,0.526192,0.532208,0.568026


# LinCKA

In [10]:
cka_d = torch.load(res_fname["lincka"])
df = pd.DataFrame(cka_d)

In [11]:
network = "bert_large_cased_0"
df.sort_values(by=network)[network]

bert_large_cased_23                 0.354745
calypso_transformer_6_512_base_6    0.355558
bert_large_cased_21                 0.361215
bert_large_cased_24                 0.367524
bert_large_cased_22                 0.375668
bert_base_cased_12                  0.405538
openai_transformer_12               0.414044
elmo_4x4096_512_0                   0.429182
openai_transformer_4                0.429187
calypso_transformer_6_512_base_2    0.437742
openai_transformer_3                0.439868
calypso_transformer_6_512_base_3    0.451706
calypso_transformer_6_512_base_5    0.452954
elmo_4x4096_512_3                   0.462159
bert_large_cased_20                 0.464681
bert_large_cased_18                 0.469123
calypso_transformer_6_512_base_4    0.470155
elmo_4x4096_512_2                   0.470948
bert_base_cased_11                  0.472110
openai_transformer_11               0.472717
elmo_4x4096_512_4                   0.474391
bert_large_cased_19                 0.477306
elmo_origi

# CCA

In [12]:
cca_d = torch.load(res_fname["cca"])

In [13]:
corrs, sv_similarities, pw_similarities = cca_d['corrs'], cca_d['sv_similarities'], cca_d['pw_similarities']

In [14]:
sv_sim_df = pd.DataFrame(sv_similarities)
pw_sim_df = pd.DataFrame(pw_similarities)
corrs_df = pd.DataFrame(corrs)
maxcorrs_df = corrs_df.applymap(lambda t: float('nan') if type(t) is float else t.max().item())

### SVCCA

In [15]:
network = "bert_large_cased_0"
sv_sim_df.sort_values(by=network)[network]

elmo_4x4096_512_4                   0.277943
openai_transformer_12               0.291095
calypso_transformer_6_512_base_6    0.291605
elmo_original_2                     0.291956
bert_large_cased_24                 0.297135
openai_transformer_11               0.305676
bert_base_cased_12                  0.315521
bert_large_cased_23                 0.316317
calypso_transformer_6_512_base_5    0.316620
openai_transformer_10               0.318087
bert_large_cased_22                 0.321090
elmo_4x4096_512_3                   0.321376
bert_large_cased_21                 0.324047
bert_large_cased_20                 0.326096
openai_transformer_9                0.326714
bert_large_cased_19                 0.328506
bert_base_cased_11                  0.329231
bert_large_cased_18                 0.334123
openai_transformer_8                0.336102
bert_base_cased_10                  0.339445
bert_large_cased_17                 0.345119
calypso_transformer_6_512_base_4    0.348826
openai_tra

### PWCCA

In [16]:
network = "bert_large_cased_0"
pw_sim_df.sort_values(by=network)[network]

elmo_4x4096_512_4                   0.370091
elmo_original_2                     0.384753
openai_transformer_12               0.388729
calypso_transformer_6_512_base_6    0.396797
bert_large_cased_24                 0.406230
openai_transformer_11               0.407604
elmo_4x4096_512_3                   0.415548
openai_transformer_10               0.422016
bert_large_cased_23                 0.427605
calypso_transformer_6_512_base_5    0.428445
openai_transformer_9                0.431988
bert_base_cased_12                  0.432118
bert_large_cased_22                 0.434581
bert_large_cased_21                 0.439737
openai_transformer_8                0.442072
bert_large_cased_20                 0.444810
bert_base_cased_11                  0.448078
bert_large_cased_19                 0.449919
elmo_4x4096_512_2                   0.451982
bert_large_cased_18                 0.457278
openai_transformer_7                0.459244
bert_base_cased_10                  0.459536
calypso_tr